In [1]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader


## Load data

In [2]:
docs="/teamspace/studios/this_studio/retrieval-augmented-generation/data/"
loader = DirectoryLoader(docs, glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

## Split document into nodes

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [4]:
len(texts)

83

In [5]:
texts[3]

Document(page_content="get to the next round of funding Beyond just that product Market fit so product Market fit is just one step just one\nstep along the way so the the challenge is something we call the The Product Company Gap and I\nfigured I'd you know since I I actually started a company I figured I'd tell you about a company that actually failed to get across that Gap um and that might be\nuseful I started a company called padient um it was a mobile payments company QR code payments before it was\nhappening in China and Korea like we actually had to write we had to code our own QR Code Reader we hired a satellite\nimagery engineer to build our own QR Reader from scratch the idea was hey we\ncould use QR codes instead of credit cards to pay for stuff um we knew like we had we started\ncompanies before and we knew that it was going to be hard to build a company a direct a consumer company how are you", metadata={'source': '/teamspace/studios/this_studio/retrieval-augmented-generat

In [ ]:
# Start Here - New Code

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores.pinecone import Pinecone
from langchain.chains import RetrievalQA
from langchain import OpenAI
from langchain_openai import ChatOpenAI
from cleantext import clean
import os
import nltk
import pinecone
import openai

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [7]:
urls = []
n_weblinks = int(input("How many web links you want the RAG agent to refer for response generation & insights? Enter here: "))
print("Enter your links below: ")
for i in range(0, n_weblinks):
  inp = input()
  # appending the element in list
  urls.append(inp)

Enter your links below: 


In [8]:
loader = UnstructuredURLLoader(urls=urls)
urls = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
# Merging urls into a single list
documents = []
documents.extend(urls)

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(documents)
texts

[Document(page_content='Skip to content\n\nMemberships\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\tClose Memberships\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\tOpen Memberships\n\nAccreditations\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\tClose Accreditations\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\tOpen Accreditations\n\nContinuous Learning\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\tClose Continuous Learning\n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\tOpen Continuous Learning\n\nCorporate Trainings\n\nContact\n\nIndividual Membership', metadata={'source': 'https://adasci.org/'}),
 Document(page_content='Join the world’s leading Data Science professional community. You can access both General & Premium Memberships.\n\nLearn More\n\nCorporate Membership\n\nAny corporate, organization or academic institution having common interests in 

In [11]:
clean_url_text = clean(text=texts,
            fix_unicode=True,
            to_ascii=True,
            lower=True,
            no_line_breaks=False,
            no_urls=False,
            no_emails=False,
            no_phone_numbers=False,
            no_numbers=False,
            no_digits=False,
            no_currency_symbols=False,
            no_punct=False,
            replace_with_punct="",
            replace_with_url="This is a URL",
            replace_with_email="Email",
            replace_with_phone_number="",
            replace_with_number="123",
            replace_with_digit="0",
            replace_with_currency_symbol="$",
            lang="en"
            )
clean_url_text

'[document(page_content=\'skip to content\nmemberships\nclose memberships\nopen memberships\naccreditations\nclose accreditations\nopen accreditations\ncontinuous learning\nclose continuous learning\nopen continuous learning\ncorporate trainings\ncontact\nindividual membership\', metadata={\'source\': \'https://adasci.org/\'}), document(page_content=\'join the world\'s leading data science professional community. you can access both general & premium memberships.\nlearn more\ncorporate membership\nany corporate, organization or academic institution having common interests in the ai field can become a member of adasci.\nlearn more\nchartered data scientist™️\nthe chartered data scientist (cds) credential gives a strong understanding of advanced data science profession and in-depth, applied analytics skills.\nlearn more\', metadata={\'source\': \'https://adasci.org/\'}), document(page_content=\'learn more\ncertified data scientist - associate level\nbest suitable for the aspirants who wa

In [12]:
from pinecone import Pinecone

# configure client
pc = Pinecone()

In [13]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [14]:
import time
index_name = "myindex"

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# we create a new index
pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='dotproduct',
        spec=spec
    )

# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [24]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 14}},
 'total_vector_count': 14}

In [16]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)


In [17]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
        texts,
        index_name=index_name,
        embedding=embeddings
    )

In [18]:
query = "What is AdaSci?"
vectorstore.similarity_search(query)

[Document(page_content='Join the world’s leading Data Science professional community. You can access both General & Premium Memberships.\n\nLearn More\n\nCorporate Membership\n\nAny corporate, organization or academic institution having common interests in the AI field can become a member of ADaSci.\n\nLearn More\n\nChartered Data Scientist™\n\nThe Chartered Data Scientist (CDS) credential gives a strong understanding of advanced data science profession and in-depth, applied analytics skills.\n\nLearn More', metadata={'source': 'https://adasci.org/'}),
 Document(page_content='Read all our Blogs\n\nOur Upcoming Conference\n\nDeep Learning DevCon 2024\n\nJoin us to hear some of the leading professionals & researchers that are pushing the boundaries of this very interesting area.\n\nRegister\n\nNot a member, but still want to know what we are upto? Subscribe to our Newsletter\n\nStart Free Trial\n\nThe power of intelligence to propel humanity and make a difference\n\nOur Accrediations\n\n

In [20]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
# completion llm
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
qa.run(query)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'ADaSci stands for the Association of Data Scientists. It is a professional community that offers memberships, accreditations, certifications, and training in the field of data science and artificial intelligence. ADaSci aims to advance AI expertise and provide global recognition for professionals in the industry.'

In [21]:
initial_prompt = "You are a researcher who is going to search the web links, summarize them and share insights as asked"

In [22]:
query = "What is the primary goal of AdaSci?"
result = qa({"query": query, "prompt": initial_prompt})

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [23]:
print(result['result'])

The primary goal of AdaSci is to advance the field of data science and artificial intelligence by providing professional certifications, training, and a community for individuals and organizations interested in these areas.
